In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.2 which is incompatible.
tensorflow 2.17.0 requires protobuf!

In [2]:
import os
import textwrap
import glob
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = "Enter the groq api key here"
os.environ["LLAMA_CLOUD_API_KEY"] = "Enter the llam api key here"


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [3]:
# instruction = """The provided document contains question-answer pairs related to standardized psychometric tools available to assess
# the psychological state of women from conception to postpartum that
# are validated for use in the Indian subcontinent.
# Try to be precise while answering the questions"""

parser = LlamaParse(
    result_type="markdown",
    # parsing_instruction=instruction,
    max_timeout=5000,
)


llama_parse_documents = await parser.aload_data("./data/Rajasthan Engineering Admission Process (REAP)-2021 _ JEC Jaipur.pdf")
print(type(llama_parse_documents))

Started parsing the file under job_id d66f6b84-7c10-4e4c-8b75-e711a9ce9153
<class 'list'>


In [4]:
parsed_doc = llama_parse_documents[0]
type(parsed_doc)

llama_index.schema.Document

In [5]:
Markdown(parsed_doc.text[:8000])

# Rajasthan Engineering Admission Process (REAP)-2021 | JEC Jaipur

+91-8875071333 tel:+918875071333 ✉ admission@jeckukas.org.in

Admission Enquiry Open 2024 | Apply for Admission

JEC 360 Virtual Campus Tour | Grievance Redressal Form

# REAP-2024

Estd: Yr 2000 | Reap Code: 1019 | Approved By AICTE | Affiliated to RTU; Kota

REAP Code: 1019 | University Results: 93%+ | Courses on offer: 36 | Departments: 8

# GENERAL INSTRUCTIONS OF RAJASTHAN ENGINEERING ADMISSION PROCESS (REAP) 2021

Following general instructions and procedures for REAP-2021 are circulated by virtue of powers entrusted through letter No. 20(1) T.E./2011 Part-II Jaipur dated 31 Jan, 2021 of Technical Education Department, Government of Rajasthan, Jaipur.

# GENERAL INSTRUCTIONS

The candidates are advised to carefully establish their eligibility before applying for B.E. / B.Tech. courses through REAP-2021. The candidate would be solely responsible for the consequences or any damage/loss/harm which occur due to wrong or erroneous facts/data/information given by the candidate.

# 1. FEE FOR REAP-2021 APPLICATION CUM REGISTRATION:

The fee for REAP-2021 Application cum Registration is Rs. 250/- (Rupees Two Hundred Fifty only) to be paid online through REAP-2021 web portal. This amount deposited for REAP-2021 is neither refundable nor transferable nor adjustable.

# 2. ONLINE SUBMISSION OF APPLICATION FORM:

2.1 REAP-2021 application form is to be submitted online on the web portal. The candidate must carefully fill all the details as required in the online application cum registration form.

2.2 For B.E. / B.Tech. courses the duration of submission of the online application form will be from 30/07/2021 to 21/08/2021. However, the last date for depositing the fees (Application cum Registration Fee) through REAP web portal will be 20/08/2021. For details of schedule of activities visit REAP portal (www.cegreap2020.com).

2.3 For B.Arch. course duration of submission of the online application form will be from 21/09/2021 to 5/10/2021. However, the last date for depositing the fees (Application cum Registration Fee) through REAP web portal will be 3/10/2021. For details of schedule of activities visit REAP portal (www.cegreap2020.com).

# 2.4 Counseling Rounds:

There would be three main rounds of the counseling in REAP-2021 except management quota admission.

In the first round, candidates from out of Rajasthan state and candidates belonging to Ex Serviceman, PwD and KM category will be given allotment of seats based on merit list.

In the second round (Upward Movement) for Candidates of first round (candidates from out of Rajasthan, Ex Serviceman, PwD and KM category) [(a) Who had their choice exhausted in the first round of allotment OR (b) those who had reported in the allotted institute and opted for Upward movement through REAP-2021 web portal] would be given Upward movement allotment and Candidates from the Rajasthan state (having Rajasthan Domicile) would be given allotment of seats based on merit list.

In the third round (Upward Movement), Candidates from the state of Rajasthan [(a) Who had their choice exhausted in the second round of allotment OR (b) those who had reported in the allotted institute and opted for Upward movement through REAP-2021 web portal] would be given Upward movement allotment.

The candidates who do not report in the allotted institute after allotment of seat are not eligible for “Upward movement”.

# 2.5 Mobile Number and Email ID:

Candidates are advised to fix their mobile number and email ID before applying in REAP-2021. All the registration and login process for students are OTP based, which will be sent on your registered mobile number and email ID. Candidates should not change their mobile number and email ID during the process of counseling/admission in REAP-2021 as all the communication would be made by the REAP-2020 office on the registered mobile number and email ID only. No postal communication would be done. Candidates are advised not to disclose their login id and password of REAP-2021 web portal to anybody. REAP-2020 will not be responsible for any loss or damage or harm caused due to candidate disclosing his/her login ID/password to any other person/institute/agency/society by either ignorance or negligence.

# 2.6 Eligibility for Admission:

# 2.6.1 Eligibility criterion for admission in first year of B.E. / B.Tech:

The minimum academic qualification for admission in B.E. /B.Tech. program in Rajasthan is Passed with at least 45% marks (40% marks in case of candidates belonging to SC/ST/Non “creamy layer OBC/ Non creamy layer MBC and PwD category” of Rajasthan State) in the final examination of class 12th (10+2).
---
# Rajasthan Engineering Admission Process (REAP)-2021 | JEC Jaipur

# Eligibility Criteria

Passed 10+2 examination with Physics and Mathematics as compulsory subjects along with one of the following subjects:

- Chemistry
- Biotechnology
- Technical Vocational subject
- Computer Science
- Information Technology
- Informatics Practices
- Agriculture
- Engineering Graphics
- Business Studies
- Biology

OR

Passed Diploma in Engineering and Technology examination from Institutions and Boards, recognized by Central or State Government / Universities with at least 45% marks (40% marks in case of candidates belonging to “SC/ST/Non creamy layer OBC/ Non creamy layer MBC and PwD category” of Rajasthan State) subjected to vacancies in the First Year, in case the vacancies at lateral entry are exhausted.

# 2.7 Priority of admission:

If the candidates having equal scaled percentage/ percentile in class 12th Science, the merit of such candidates shall be decided in the order mentioned under:

1. Resolving by marks obtained in Mathematics in class 12th Science, candidate having higher marks percentage will be given priority.
2. Resolving by marks obtained in physics in class 12th Science, candidate having higher marks percentage will be given higher (better) rank.
3. Resolving by Date of Birth of candidate, candidate having older Date of Birth will be given higher (better) rank.
4. Resolving by Form number (Lower Form number will be given priority/better rank i.e. First applied first served).

# 2.7.1 Priorities of Admission for B.E./B.Tech. courses:

Candidate is required to register through REAP-2021 web portal. Eligible candidate may register on the basis of class 12th Science or Diploma approved by state govt. in Engineering Technology. However, Scaled Percentage/Percentile of class 12th Science would be given higher priority in comparison to percentage of Diploma in Engineering & Technology approved by state government and AICTE in preparing the combined merit list during all the stages of counseling in REAP-2021 as listed below.

|S.No|Basis of admissions|Priority|
|---|---|---|
|1|Scaled Percentage/Percentile of class 12th Science|1st|
|2|Percentage of Diploma in Engineering & Technology approved by state government|2nd|

If the candidates having equal obtained marks/ percentage in Diploma in Engineering & Technology approved by state government and AICTE, the merit of such candidates shall be decided in the order mentioned under:

1. Resolving by Date of Birth of candidate, candidate having older Date of Birth will be given higher (better) rank.
2. Resolving by Form number (Lower Form number will be given priority/better rank i.e. First applied first served).

# 2.8 Admission in B.E./B.Tech. and B.Arch. courses under out of state quota:

Candidates who belong to any State other than Rajasthan shall be considered for admission in first year of B.E./B.Tech. and B.Arch. courses. 15% seats of approved intake (For private engineering institutions and self-financed category in the Autonomous Engineering Colleges of Govt. of Rajasthan as mentioned in SEAT MATRIX are available for such candidates).

If seats remain vacant (Unfilled) after completion of admission in this round (including upward movement of eligible candidates) the va

In [6]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [7]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

8

In [9]:
print(docs[0].page_content)


Rajasthan Engineering Admission Process (REAP)-2021 | JEC Jaipur

+91-8875071333 tel:+918875071333 ✉ admission@jeckukas.org.in

Admission Enquiry Open 2024 | Apply for Admission

JEC 360 Virtual Campus Tour | Grievance Redressal Form

REAP-2024

Estd: Yr 2000 | Reap Code: 1019 | Approved By AICTE | Affiliated to RTU; Kota

REAP Code: 1019 | University Results: 93%+ | Courses on offer: 36 | Departments: 8

GENERAL INSTRUCTIONS OF RAJASTHAN ENGINEERING ADMISSION PROCESS (REAP) 2021

Following general instructions and procedures for REAP-2021 are circulated by virtue of powers entrusted through letter No. 20(1) T.E./2011 Part-II Jaipur dated 31 Jan, 2021 of Technical Education Department, Government of Rajasthan, Jaipur.

GENERAL INSTRUCTIONS

The candidates are advised to carefully establish their eligibility before applying for B.E. / B.Tech. courses through REAP-2021. The candidate would be solely responsible for the consequences or any damage/loss/harm which occur due to wrong or erro

In [10]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

In [11]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [12]:
%%time
query = "tell me about the admission procedure"
similar_docs = qdrant.similarity_search_with_score(query)
len(similar_docs)

CPU times: user 224 ms, sys: 805 µs, total: 225 ms
Wall time: 228 ms


4

In [13]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:2048]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: In case of transaction failure applies in the prescribed format and sends the Transaction Failure form to email ID: helpinadmissions@gmail.com.

10. DEPOSITION OF FEES AND ORIGINAL DOCUMENTS:

At the time of reporting in the allotted institute, the candidates will have to show all original documents, certificates and receipt of registration cum application form fee along with provisional seat allotment letter generated by REAP-2021 web portal.

The candidate is also required to deposit a self-attested copy of all the documents. The institute admission authority of the allotted/participating institute would thoroughly check the documents and other required certificates.

The designated admission authority of the allotted/participating institute would make arrangements for Have Any Query? Chat with us!

https://jeckukas.org.in/Admission/REAP

Rajasthan Engineering Admission Process (REAP)-2021 | JEC Jaipur

The online reporting of the candidate at the institute. The candidate shoul

In [14]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
# retrieved_docs = retriever.invoke(query)

CPU times: user 94 µs, sys: 8 µs, total: 102 µs
Wall time: 104 µs


In [15]:

# for doc in retrieved_docs:
#     print(f"id: {doc.metadata['_id']}\n")
#     print(f"text: {doc.page_content[:256]}\n")
#     print("-" * 80)
#     print()


In [16]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:01<00:00, 22.6MiB/s]


In [17]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.07 s, sys: 96.9 ms, total: 2.17 s
Wall time: 2.16 s


3

In [18]:
# for doc in reranked_docs:
#     print(f"id: {doc.metadata['_id']}\n")
#     print(f"text: {doc.page_content[:1024]}\n")
#     print(f"score: {doc.metadata['relevance_score']}")
#     print("-" * 80)
#     print()

In [19]:
llm = ChatGroq(temperature = 0, model_name="llama3-70b-8192")

In [20]:
prompt_template = """
Given the Context below, generate a response that addresses the Question. Assume the context represents information related to engineering and polytechnic colleges under the Department of Technical Education, Government of Rajasthan. The information may include admission processes, eligibility criteria, college details, fee structures, scholarship opportunities, hostels, historical allotments, curriculum, and placements.

Your response should aim to:

Efficiently Retrieve Information: Quickly provide accurate and relevant details from the CONTEXT.
Ensure Accessibility: Use Natural Language Processing to enhance understanding, and provide a concise, easy-to-follow response.
Be User-Friendly: Respond in a conversational tone that is accessible and intuitive for students, parents, and stakeholders.
Capture User Needs: If the QUESTION is not fully addressed in a single response, guide the user on what other information they may need or what to do next.
Optimize in English, with future capabilities for Hindi and other regional languages.
Input Format:

Context: {context}
Question: {question}
Example Input:

CONTEXT: "The admission process involves three steps: (1) Online registration, (2) Document verification, (3) Seat allocation based on merit. Fees for the registration are INR 500 for general students and INR 250 for reserved categories. Scholarships are available for economically weaker sections."
QUESTION: "What is the admission process and are there any scholarships available?"
Expected Output Format: Provide a well-structured response that:

Explains the admission process clearly.
Mentions the scholarship opportunities.
Uses natural, conversational language to engage the user."""


prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [21]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)


In [43]:
%%time
print_response(qa.invoke(input("Enter your query: ")))
#INPUT THE QUESTION HERE

Enter your query: What all documents do I require for admission process
Running pairwise ranking..
Here is the response to the question:

"Hey there! For the admission process, you'll need to carry the following documents:

1. Printout of provisional seat allotment letter taken from the REAP-2021 web portal using your
login details.
2. Aadhar Card/Aadhar Acknowledgement, Receipt (if available).
3. Class X (High School) Board Mark sheet/Certificate as proof of date of birth.
4. Mark sheet of class 12th Science (for Subjects) or Diploma.
5. Certificate of category (SC/ST/OBC/MBC/EWS), issued by the competent authority (if applicable).
6. Undertaking by OBC/MBC in required format (for non-creamy layer), (if applicable).
7. Certificate for Persons with Disabilities (PwD), (if applicable).
8. Certificate for Ex-Service men, (if applicable).
9. Domicile certificate.
10. Income Certificate.
11. Medical Fitness Certificate.
12. Fee to be deposited at the Institute.

Remember to deposit the req

In [23]:
 # import os
# import glob
# import asyncio
# from llama_parse import LlamaParse  # Replace with the actual import statement for LlamaParse
# from unstructured.markdown import UnstructuredMarkdownLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import FastEmbedEmbeddings
# from qdrant_client import Qdrant

# # Additional import if required for file I/O
# import os.path


In [24]:
# import os.path
# parser = LlamaParse(
#     result_type="markdown",
#     # parsing_instruction=instruction,  # Uncomment if you have specific instructions
#     max_timeout=10000,
# )

In [25]:
# async def main():
#     # Directory where PDF files are located
#     pdf_directory = './data'
#     # Directory to save markdown files
#     markdown_directory = './data'
#     # Path to save or load the Qdrant database
#     qdrant_db_path = './db'

#     # Initialize lists to collect documents
#     all_docs = []

#     # Get all PDF files in the directory
#     pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

#     for pdf_file in pdf_files:
#         # Parse the PDF to markdown
#         markdown_content = await parser.aload_data(pdf_file)

#         # Save the markdown content to a .md file
#         markdown_file = os.path.join(markdown_directory, os.path.basename(pdf_file).replace('.pdf', '.md'))
#         with open(markdown_file, 'w') as md_file:
#             md_file.write(markdown_content[0])

#         # Load the markdown content using UnstructuredMarkdownLoader
#         loaded_documents = UnstructuredMarkdownLoader(markdown_file)

#         # Split the documents using text splitter
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
#         docs = text_splitter.split_documents(loaded_documents)

#         # Collect all documents
#         all_docs.extend(docs)

#     # Create an embeddings object
#     embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

#     # Check if Qdrant database already exists
#     if os.path.exists(qdrant_db_path):
#         # Load existing Qdrant database
#         qdrant = Qdrant(location=qdrant_db_path)
#     else:
#         # Create a new Qdrant database
#         qdrant = Qdrant(location=":memory:")  # Temporarily in memory to initialize
#         qdrant.save(path=qdrant_db_path)      # Save the new database

#     # Add new documents to the Qdrant database
#     qdrant.add_documents(
#         docs,
#         embeddings,
#         collection_name="document_embeddings",
#     )

#     # Save the updated Qdrant database
#     qdrant.save(path=qdrant_db_path)


In [26]:
# # Directory where PDF files are located
# pdf_directory = './data'
# # Directory to save markdown files
# markdown_directory = './data'
# # Path to save or load the Qdrant database
# qdrant_db_path = './db'

# # Initialize lists to collect documents
# all_docs = []

# # Get all PDF files in the directory
# pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

# for pdf_file in pdf_files:
#     # Parse the PDF to markdown
#     markdown_content = await parser.aload_data(pdf_file)

#     # Save the markdown content to a .md file
#     markdown_file = os.path.join(markdown_directory, os.path.basename(pdf_file).replace('.pdf', '.md'))
#     with open(markdown_file, "a") as md_file:
#         md_file.write(markdown_content[0].text)

#     # Load the markdown content using UnstructuredMarkdownLoader
#     loader = UnstructuredMarkdownLoader(markdown_file)
#     loaded_documents = loader.load()

#     # Split the documents using text splitter
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
#     docs = text_splitter.split_documents(loaded_documents)

#     # Collect all documents
#     all_docs.extend(docs)

In [27]:
# import nest_asyncio
# nest_asyncio.apply()
# from httpx import ReadTimeout

# # Retry parameters
# max_retries = 3
# retry_delay = 5  # in seconds

# async def process_pdf(pdf_file):
#     retries = 0
#     while retries < max_retries:
#         try:
#             print(f"Processing file: {pdf_file} (Attempt {retries + 1}/{max_retries})")

#             # Parse the PDF to markdown
#             markdown_content = await parser.aload_data(pdf_file)
#             print(f"Successfully parsed PDF: {pdf_file}")

#             # Save the markdown content to a .md file
#             markdown_file = os.path.join(markdown_directory, os.path.basename(pdf_file).replace('.pdf', '.md'))
#             with open(markdown_file, "w") as md_file:  # Changed "a" to "w" to overwrite if exists
#                 md_file.write(markdown_content[0].text)
#             print(f"Markdown file saved: {markdown_file}")

#             # Load the markdown content using UnstructuredMarkdownLoader
#             loader = UnstructuredMarkdownLoader(markdown_file)
#             loaded_documents = loader.load()
#             print(f"Loaded documents from markdown file: {markdown_file}")

#             # Split the documents using text splitter
#             text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
#             docs = text_splitter.split_documents(loaded_documents)
#             print(f"Successfully split documents for file: {pdf_file}")

#             return docs

#         except ReadTimeout:
#             print(f"ReadTimeout occurred while processing {pdf_file}. Retrying {retries + 1}/{max_retries}...")
#             retries += 1
#             await asyncio.sleep(retry_delay)
#         except Exception as e:
#             print(f"An error occurred while processing {pdf_file}: {e}")
#             retries += 1
#             await asyncio.sleep(retry_delay)

#     print(f"Failed to process {pdf_file} after {max_retries} retries.")
#     return []

# # Check if an event loop is already running
# if asyncio.get_event_loop().is_running():
#     # Create tasks and await them directly
#     tasks = [process_pdf(pdf_file) for pdf_file in pdf_files]
#     results = await asyncio.gather(*tasks)
# else:
#     # Create tasks and run them using asyncio.run()
#     tasks = [process_pdf(pdf_file) for pdf_file in pdf_files]
#     results = asyncio.run(asyncio.gather(*tasks))

# # Flatten the list of results
# for result in results:
#     all_docs.extend(result)

# # The all_docs list now contains all the processed documents

In [28]:
# pdf_files

In [29]:
# embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [30]:
# if os.path.exists(qdrant_db_path):
#     # Load existing Qdrant database
#     from qdrant_client import QdrantClient
#     client = QdrantClient(path = qdrant_db_path)
#     db = Qdrant(client=client, collection_name="document_embeddings", embeddings=embeddings)

#     qdrant.add_documents(
#       all_docs,
#       embeddings,
#       collection_name="document_embeddings",
#     )

#     qdrant.save(path=qdrant_db_path)
# else:
#     qdrant = Qdrant.from_documents(
#     all_docs,
#     embeddings,
#     # location=":memory:",
#     path="./db",
#     collection_name="document_embeddings",
# )

# # Save the updated Qdrant database

In [31]:
# qdrant = Qdrant.from_documents(
#     docs,
#     embeddings,
#     location=":memory:",
#     path="./db",
#     collection_name="document_embeddings",
# )

In [32]:
# all_docs

In [33]:
# document_path = Path("data/parsed_document.md")
# with document_path.open("a") as f:
#     f.write(parsed_doc.text)
# #
# loader = UnstructuredMarkdownLoader(document_path)
# loaded_documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
# docs = text_splitter.split_documents(loaded_documents)
# len(docs)